In [1]:
#librerias a usar
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
#leer el archivo json

chunks = pd.read_json('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review.json', lines=True, chunksize=1000)

df_review_crudo = pd.concat(chunks)

In [1]:
#Analizar el 'peso' del archivo

import os

size_in_bytes = os.path.getsize('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review.json')
size_in_megabytes = size_in_bytes / 1e6

print(f'La capacidad del archivo "review.json" es de {size_in_megabytes} MB')

La capacidad del archivo "review.json" es de 5341.868833 MB


In [3]:
df_review_crudo.shape

(6990280, 9)

In [4]:
df_review_crudo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [8]:
#hacer un csv con las primeras 100 filas para ver la estructura del archivo
df_review_crudo.head(100).to_csv('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/df_review_crudo_100_filas.csv', index=False)

In [5]:
#verificar si hay valores nulos
df_review_crudo.isnull().sum()


review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [9]:
#verificar si hay valores duplicados
df_review_crudo.duplicated().sum()



0

### Analisis de las columnas: 

Segun el estudio realizado, nos encontramos con tres columnas en particular que se escapan a nuestro objetivo, las mismas son: 

<u>useful (int64):</u> Es el número de veces que otros usuarios han marcado la reseña como "útil". Este valor es un número entero.<span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

<u>funny (int64):</u>  Es el número de veces que otros usuarios han marcado la reseña como "divertida". Este valor es un número entero.<span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

<u> cool (int64): </u> Es el número de veces que otros usuarios han marcado la reseña como "genial" o "interesante". Este valor es un número entero.<span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

In [3]:
#hacemos una copia del dataframe original
df_review = df_review_crudo.copy()

In [4]:
#verificamos la copia
df_review.shape

(6990280, 9)

In [5]:
#y dropeamos las columnas que de 'useful', 'funny' y 'cool' que no vamos a usar
df_review.drop(columns=['useful', 'funny', 'cool'], inplace=True)

In [6]:
#verificamos que se hayan dropeado
df_review.shape

(6990280, 6)

In [7]:
#guardamos el df en un nuevo archivo parquet
df_review.to_parquet('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review_listo_para_usar.parquet', index=False)



In [8]:
#Analizamos el 'peso' del nuevo archivo parquet
import os

size_in_bytes = os.path.getsize('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review_listo_para_usar.parquet')
size_in_megabytes = size_in_bytes / 1e6

print(f'La capacidad del archivo parquet "review_listo_para_usar" es de {size_in_megabytes} MB')

La capacidad del archivo parquet "review_listo_para_usar" es de 2959.282368 MB


In [9]:
import os

# Obtén el tamaño del archivo JSON
size_in_bytes_json = os.path.getsize('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review.json')
size_in_megabytes_json = size_in_bytes_json / 1e6

# Obtén el tamaño del archivo Parquet
size_in_bytes_parquet = os.path.getsize('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review_listo_para_usar.parquet')
size_in_megabytes_parquet = size_in_bytes_parquet / 1e6

# Calcula la diferencia
diferencia = size_in_megabytes_json - size_in_megabytes_parquet 

print(f'La diferencia de peso entre el archivo json y el parquet es de {diferencia} MB')



La diferencia de peso entre el archivo json y el parquet es de 2382.5864650000003 MB


In [1]:
df_review.columns

NameError: name 'df_review' is not defined